In [1]:
from dotenv import load_dotenv
load_dotenv(".env")

import torch
from datasets import Dataset, load_dataset
from langchain_google_genai import ChatGoogleGenerativeAI
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import FactualCorrectness
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm

In [2]:
ds = load_dataset("rungalileo/ragbench", "hotpotqa", split="test")

In [3]:
ds[0]['question']

'Which university did one of the key figures in the American documentary film, released in 2015, directed by Malcolm Ingram, pay for, before being drafted 18th overall pick for the New Jersey Nets?'

In [4]:
evaluator_llm = LangchainLLMWrapper(
    ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

E0000 00:00:1761038542.364265  822636 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
/tmp/ipykernel_822636/2935404805.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = LangchainLLMWrapper(


In [5]:
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# DoLa Decoding

In [6]:
num_samples = 2
dataset = []
for d in tqdm(ds.select(range(num_samples)), total=num_samples):
    question = d["question"]
    reference = d["response"]

    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        repetition_penalty=1.2,
        custom_generate="custom_decoding/dola",
        trust_remote_code=True,
        dola_layers='high',
    )

    response = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0]

    dataset.append(
        {
            "user_input": question,
            "response": response.strip(),
            "reference": reference,
        }
    )

100%|██████████| 2/2 [01:48<00:00, 54.01s/it]


In [7]:
evaluation_dataset = Dataset.from_list(dataset)

ragas_result = evaluate(evaluation_dataset, metrics=[FactualCorrectness()], llm=evaluator_llm)
print(ragas_result)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

E0000 00:00:1761038667.709127  822636 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'factual_correctness(mode=f1)': 0.3750}


In [8]:
ragas_result_df = ragas_result.to_pandas()
ragas_result_df.head()

,user_input,response,reference,factual_correctness(mode=f1)
0,Which university did one of the key figures in...,(1963-1981)\n\nThe question appears to be a ri...,One of the key figures in the American documen...,0.00
1,Were both Léopold Eyharts and Ulrich Walter a ...,1. Léopold Eyharts\n2. Ulrich Walter\n\nPlease...,"No, only Léopold Eyharts was a General in the ...",0.75


# SLED Decoding

In [9]:
num_samples = 2
dataset = []
for d in tqdm(ds.select(range(num_samples)), total=num_samples):
    question = d["question"]
    reference = d["response"]

    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        repetition_penalty=1.2,
        custom_generate="custom_decoding/sled",
        trust_remote_code=True,
        evolution_rate=2.0,
        evolution_scale=10,
        evolution_lower_bound=-1000.0,
    )

    response = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0]

    dataset.append(
        {
            "user_input": question,
            "response": response.strip(),
            "reference": reference,
        }
    )

100%|██████████| 2/2 [01:08<00:00, 34.01s/it]


In [10]:
evaluation_dataset = Dataset.from_list(dataset)

ragas_result = evaluate(evaluation_dataset, metrics=[FactualCorrectness()], llm=evaluator_llm)
print(ragas_result)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'factual_correctness(mode=f1)': 0.0000}


In [11]:
ragas_result_df = ragas_result.to_pandas()
ragas_result_df.head()

,user_input,response,reference,factual_correctness(mode=f1)
0,Which university did one of the key figures in...,The question appears to contain a mix-up or co...,One of the key figures in the American documen...,0.0
1,Were both Léopold Eyharts and Ulrich Walter a ...,1. **Léopold Eyharts** \n2. **Ulrich Walter**...,"No, only Léopold Eyharts was a General in the ...",0.0
